# HNX-M_v4 Research Notebook
## _“Public Research Release”_

**A Dual-Strand, Entropy-Gated Neural Architecture for Memory-Efficient Sequence Learning**

This notebook documents the architecture, biological inspiration, performance results, and patent claims for the HNX-M_v4 model.

> **Note:** This repository contains partial implementation and research results for the patented HNX-M architecture. Core implementation details are proprietary and omitted.

## 1. Introduction

Modern sequence models such as Transformers, GRUs, and Mamba-based SSMs have improved performance across many domains but still face challenges:
- Stability over long sequences
- Efficient memory use
- Parameter efficiency
- Fast convergence

HNX-M is a biologically inspired architecture that addresses these issues by combining:
- **Dual-strand processing** (forward + backward)
- **Shared memory ladder** with learnable rungs
- **Entropy-gated modulation** for selective recall
- **Asymmetric projection** for strand specialisation

## 2. Architecture Overview

### Structural Features – Performance Mapping

| Structural Feature                          | Technical Mechanism                                                                 | Primary Impact Area                      | Observed Outcome (Test 10)                                   |
|--------------------------------------------|-------------------------------------------------------------------------------------|------------------------------------------|--------------------------------------------------------------|
| Dual Strand (Forward & Backward)         | Causal conv1d + reversed scan w/ entropy gating                                     | Bidirectional memory + recurrence        | Faster convergence; handles asymmetric temporal cues         |
| Memory Ladder (LearnableDecayMemory)     | Slot-based memory w/ learned decay + Δt bias                                        | Long-term storage & selective recall     | Stable memory access across sequence; no performance dip     |
| Gated Memory Injection                   | `sigmoid(mem_gate(fwd_out)) * memory_read`                                         | Precision of memory recall               | Smoother integration of memory; avoids overwriting signal    |
| Entropy-Gated Backward Scaling           | Entropy-to-scalar gate modulates backward strand                                   | Backward recall relevance                | Prevents noisy backward signals from derailing output        |
| Slot Projection (Asymmetric)             | Separate `slot_proj` and `slot_proj_bwd`                                            | Write/read disentanglement               | Enables divergent memory focus per strand                    |
| Per-Slot Write Strengths (NEW)           | `MLP([entropy, delta]) → strength * update`                                        | Write modulation per slot                | Best performance to date; stable loss & highest accuracy     |
| Positional Biasing via Learned Δt        | `dt_bias_fwd`, `dt_bias_bwd` injected into positional encoding                     | Temporal pattern sensitivity              | Removes jitter mid-training; helps sustain low final loss    |
| Torsion Gate Projection                  | `project_with_torsion_split` with gating via `sigmoid(gate)`                       | Signal sharpening at input stage         | Improves convergence curve shape; reduces early noise        |
| Fusion Weights (Learned)                 | Learnable `[fwd, bwd, mem]` vector → weighted sum before output                     | Adaptive decoding                        | Balances pathways dynamically across sequence                |
| Rung Signal (Emergent)                   | `compute_state_delta` → `compute_rung_signal`                                       | Memory write trigger granularity         | Still used in logging; not yet re-introduced as gate trigger  |

### Biological Analogies

| Structural Feature                          | Biological Analogy                          | Interpretation / Role in System                                                    |
|--------------------------------------------|---------------------------------------------|-------------------------------------------------------------------------------------|
| Dual Strand (Forward & Backward)         | DNA double helix / bidirectional axons      | Dual propagation pathways for sensing + integration (afferent/efferent signals)    |
| Memory Ladder (LearnableDecayMemory)     | Synaptic weight traces / long-term memory   | Selectively reinforced memories with decay → synaptic plasticity                   |
| Gated Memory Injection                   | Neurotransmitter gating at dendrites        | Controls which memory traces influence output — like receptor/channel modulation   |
| Entropy-Gated Backward Scaling           | Attention focus during cognitive load       | Filters irrelevant memories in high-uncertainty states — like cognitive fatigue    |
| Slot Projection (Asymmetric)             | Functional lateralization (brain hemispheres) | Read vs write asymmetry enables diversified representation                         |
| Per-Slot Write Strengths                 | Hebbian learning with localized adaptation  | Each "neuron" adapts differently to learning signals — slot-level plasticity        |
| Positional Biasing via Learned Δt        | Circadian / neural oscillation entrainment  | Aligns processing to internal temporal rhythms — like entrained signal windows     |
| Torsion Gate Projection                  | Protein folding via torsional control       | Reshapes signal pathways at input to guide folding/activation                      |
| Fusion Weights (Learned)                 | Hormonal modulation / region weight tuning  | Dynamically shifts emphasis between memory, current input, and retrospection       |
| Rung Signal (Emergent)                   | Calcium spike / dendritic activation burst  | Triggers when change is high — proxy for firing threshold being crossed            |

## 3. Code Walkthrough — HNX-M

![Architecture Diagram](../figures/architecture_diagram.png)

> Diagram illustrating dual strands, shared memory ladder, and gating components.

### Code cell 1 - Imports & Class Definition

In [ ]:
"""
HNX-M_v4 (Public Research Release)
----------------------------------
Core `core/` module implementations omitted — covered under patent protection.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from models.metadata import init_meta_logs, update_meta_logs, finalize_meta_logs
from core.gates import EntropyGate, compute_entropy
from core.scan import StrandScanContrastProcessor
from core.rung_logic import compute_state_delta, compute_rung_signal
from core.strand import (
    StrandProcessor, LearnableDecayMemory,
    BackwardScaler, project_with_torsion_split, flip_time
)

class HNX_M_V4(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_memory_slots):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.input_proj = nn.Linear(input_dim, 2 * hidden_dim)
        self.fusion_weight = nn.Parameter(torch.ones(3))  # [fwd, bwd, mem]
        
        # Forward & backward strands
        self.forward_strand = StrandProcessor(dim=hidden_dim, direction="forward", kernel_size=2)
        self.backward_strand = StrandScanContrastProcessor(dim=hidden_dim)
        
        # Memory ladder
        self.memory = LearnableDecayMemory(num_memory_slots, hidden_dim)
        self.mem_gate = nn.Linear(hidden_dim, 1)
        
        # Gating & projection
        self.entropy_gate = EntropyGate(hidden_dim)
        self.backward_scaler = BackwardScaler()
        self.slot_proj = nn.Linear(hidden_dim, num_memory_slots)
        self.slot_proj_bwd = nn.Linear(hidden_dim, num_memory_slots)
        self.output_proj = nn.Linear(hidden_dim, output_dim)
        
        # Positional biasing
        self.dt_bias_fwd = nn.Parameter(torch.zeros(self.hidden_dim))
        self.dt_bias_bwd = nn.Parameter(torch.zeros(self.hidden_dim))


**Structural Features:**
- Dual Strand (Forward & Backward)
- Memory Ladder
- Fusion Weights (Learned)
- Positional Biasing via Δt

**Biological Analogies:**
- DNA double helix / bidirectional axons
- Synaptic weight traces
- Hormonal modulation / pathway weighting
- Circadian / neural oscillation entrainment

### Code cell 2 - Forward Method (Part 1: Input & Forward Strand)

In [ ]:
def forward(self, x):
    meta_logs = init_meta_logs()
    B, T, _ = x.shape

    # === Torsion Phase + Gate Split ===
    x = project_with_torsion_split(x, self.input_proj, self.hidden_dim, self.dt_bias_fwd)

    # === Forward Strand Encoding ===
    fwd_out = self.forward_strand(x)

**Structural Features:**
- Torsion Gate Projection — reshapes signal pathways at input.
- Dual Strand (Forward) — processes sequence in natural order.

**Biological Analogies:**
- Protein folding via torsional control.
- Afferent signal pathway in neural systems.


### Code cell 3 - Memory Rung Signal & Write Weights

In [ ]:
# === Compute Rung Signal ===
h_prev = torch.roll(fwd_out, shifts=1, dims=1)
delta = compute_state_delta(fwd_out, h_prev)
rung_signal = compute_rung_signal(delta)

# === Forward Slot Weights + Write ===
slot_logits = self.slot_proj(fwd_out)
slot_weights = torch.softmax(slot_logits, dim=-1)

# === Backward Slot Weights + Read ===
slot_weights_bwd = torch.softmax(self.slot_proj_bwd(fwd_out.detach()), dim=-1)
mem_out = self.memory.read_all(slot_weights_bwd)

**Structural Features:**
- Rung Signal (Emergent) — detects high state change to trigger memory updates.
- Slot Projection (Asymmetric) — separate forward/backward slot projections.
- Memory Ladder — selective read/write via attention over rungs.

**Biological Analogies:**
- Calcium spike / dendritic activation bursts.
- Functional lateralisation (brain hemispheres).
- Synaptic plasticity.


### Code cell 4 - Gating & Backward Strand

In [ ]:
# === Apply per-token memory gate ===
mem_gate = torch.sigmoid(self.mem_gate(fwd_out))
mem_out = mem_out * mem_gate

# === Backward Strand ===
entropy = compute_entropy(fwd_out)
entropy_val = entropy.mean(dim=-1)
gate_val = self.backward_scaler(entropy_val)

bwd_in = flip_time(fwd_out.detach())
bwd_scanned = self.backward_strand(bwd_in)
bwd_out = flip_time(bwd_scanned)
bwd_out = gate_val.unsqueeze(-1) * bwd_out

**Structural Features:**
- Gated Memory Injection — precision control of recall.
- Entropy-Gated Backward Scaling — filters backward recall when uncertainty is high.

**Biological Analogies:**
- Neurotransmitter gating at dendrites.
- Cognitive load filtering (focus under uncertainty).


### Code cell 5 - Output Projection & Return

In [ ]:
# === Output Projection ===
combined = torch.stack([fwd_out, bwd_out, mem_out], dim=0)
weighted = (self.fusion_weight[:, None, None, None] * combined).sum(dim=0)
return self.output_proj(weighted), finalize_meta_logs(meta_logs)

**Structural Features:**
- Fusion Weights (Learned) — adaptive pathway balance.
- Final output projection for token-wise predictions.

**Biological Analogies:**
- Hormonal modulation / dynamic emphasis between sensory, memory, and retrospective signals.

## 4. Patent Coverage

See the **Claims** and **Description** sections for full patent text.

### Claim 1 (Independent)
- Dual-strand (forward + backward) sequence processing
- Shared memory ladder with gated rungs
- Entropy-based modulation for memory access, positional torsion, and dropout
- Output projection combining forward, backward, and memory vectors

*(Additional dependent claims follow in patent filing)*

## 5. Experimental Setup

- **Environment:** GridWorld variants with dynamic hazards
- **Episodes:** 4000
- **Epochs:** 10
- **Hardware:** AMD RDNA3 GPU (ROCm)
- **Metrics:** Accuracy, Loss, Stability

### Test 10 Summary
- Accuracy: 58.35%
- Avg Loss: 0.8813
- Train Time: 208.8s
- Smoothest convergence yet, no regression across epochs

## 6. Results

![Loss Curve](../figures/loss_curve_test10.png)

![Rung Activation Heatmap](../figures/rung_activation_heatmap.png)

- **Observation:** Per-slot write strengths + entropy-gated backward scaling yielded the highest stability and accuracy to date.

## 7. Discussion

- Dual-strand architecture improved early convergence speed compared to baselines.
- Entropy gating prevented noisy backward signals from destabilising training.
- Asymmetric slot projection allowed strand specialisation, improving memory selectivity.
- Biological analogies suggest parallels with synaptic plasticity and hemispheric specialisation.

## 8. Conclusion & Future Work

HNX-M_v4 demonstrates that biologically inspired sequence architectures can achieve both stability and adaptability.

Future research directions:
- Mutation masking
- Temporal homeostasis
- Stochastic rewiring
- Synapse sparsity control